In [1]:
# Bokeh libraries
import bokeh
import pandas as pd
import numpy as np
import os
from bokeh.io import output_file, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Select, Slider, TextInput
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Tabs, Panel

In [2]:
path = os.listdir('./data/')

cfrm_g = pd.read_csv('./data/' + path[0] , index_col=1+0)
cfrm_reg = pd.read_csv('./data/' + path[1])
cfrm_narrow = pd.read_csv('./data/' + path[2])

death_g = pd.read_csv('./data/' + path[3], index_col=1)
death_reg = pd.read_csv('./data/' + path[4])
death_narrow = pd.read_csv('./data/' + path[5])

recov_g = pd.read_csv('./data/' + path[6], index_col=1)
recov_reg = pd.read_csv('./data/' + path[7])
recov_narrow = pd.read_csv('./data/' + path[8])

In [3]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure

from bokeh.models import CategoricalColorMapper, HoverTool, ColumnDataSource, Panel
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, RadioButtonGroup

from bokeh.layouts import column, row, WidgetBox, layout
from bokeh.palettes import Category20_16

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application
from bokeh.transform import dodge

output_notebook()
output_file("Tubes.html")

def modify_doc(doc) :
    re_index = lambda x: [x.iloc[i+1][1] if (type(x.iloc[i+1][0]) ==float) else (str(x.iloc[i+1][1])+' ('+str(x.iloc[i+1][0])+')') for i in range(len(x[x.columns[0]])-1)]
    cfrm_g.index = re_index(cfrm_reg)
    recov_g.index = re_index(recov_reg)
    death_g.index = re_index(death_reg)
    
    years = cfrm_g.columns[3:].tolist()
    
    status = ['Recovered', 'Confirm', 'Death']
    colors = [ "green", "blue", "red"]

    #Set Input
    
    slider_month = Slider(title ='Month', value=0, start=0, end=4, step=1, height=1, width=10)
    
    region = Select(title="Country/Region", value="Afghanistan",
               options= cfrm_g.index.tolist(), height=1, width=10)
    
    RB = RadioButtonGroup(labels=["Show All", "Hide Recovered", "Hide Confirm", "Hide Death"], active=0)
    
    s = [(status) for i in range(86)]
    #x = [(year) for year in years]
    x = pd.to_datetime(years)
    def select_region():
        #get input value
        region_val = region.value
        month = slider_month.value

        if slider_month.value != 0 :
            #get_month = lambda x,y : [s for s in x if int(s[0])==y]
            get_month = lambda x,y : [y for s in range(y) for y in years if int(y[0])==s+1 ]
            month = get_month(years, month)
        else :
            month = years
            
        if (region_val != 'All'):
            print(region_val)
            
            selected = dict(Confirm = cfrm_g[month].loc[region_val].tolist(),
                             Recovered = recov_g[month].loc[region_val].tolist(),
                                Death = death_g[month].loc[region_val].tolist(),
                            Month = month)
        if (RB.active != 0):
            print(RB.active)
            selected[RB.labels[RB.active].split()[1]] = np.zeros(86)
        print(selected)
        return selected
    def global_virus() :
        print('Global Diseas')
        
        month = slider_month.value
        if slider_month.value != 0 :
            get_month = lambda x,y : [y for s in range(y) for y in years if int(y[0])==s+1 ]
            month = get_month(years, month)
        else :
            month = years
            
        g_month = lambda data,months : [data[i].sum() for i in months]

        selected = dict(Confirm = g_month(cfrm_g, month),
                         Recovered = g_month(recov_g, month),
                         Death = g_month(death_g, month),
                         Month = month)
        #print("=>", selected)
        if (RB.active != 0):
            selected[RB.labels[RB.active].split()[1]] = np.zeros(86)
        
        print(selected)
        return selected
    
    #set Data
    #print('Set Data')
    data = select_region()
    data_g = global_virus()
    #print(data)
    
    TOOLTIPS=[("Number", "@$name"), ("Date ", "@x")]
    
    p = figure(x_range=years,y_range=(0,1000), 
               plot_width=300, plot_height=200,  
               title='Region %s Covid-19' %region.value, tools="crosshair,pan,reset,save,wheel_zoom",
              tooltips= TOOLTIPS)
   
    p_g = figure(x_range=years,y_range=(0,int(max(data_g['Confirm']))), 
               plot_width=300, plot_height=200,  
               title='Global Covid-19' , tools="crosshair,pan,reset,save,wheel_zoom",
              tooltips= TOOLTIPS, x_axis_type="datetime")
    
    source = ColumnDataSource(data=dict(x=years, recovered = data['Recovered'],
                                        confirmed = data['Confirm'],
                                        death= data['Death'], status=s))
    
    source_g = ColumnDataSource(data=dict(x=years, recovered = data_g['Recovered'],
                                        confirmed = data_g['Confirm'],
                                        death= data_g['Death'], status=s))
    
    p_g.line(x='x', y='recovered', line_width=2, source=source_g, legend_label="Recovered", color='green', name='recovered')
    p_g.line(x='x', y='confirmed', line_width=2, source=source_g, legend_label="Confirmed", color='blue', name='confirmed')
    p_g.line(x='x', y='death', line_width=2, source=source_g, legend_label="Death", color='red', name='death')
    
    p.vbar(x=dodge('x', -0.25, range=p.x_range), top='recovered', bottom=0,
           width=0.2, color=colors[0], source = source, name='recovered', legend_label="Recovered")
    
    p.vbar(x=dodge('x', 0., range=p.x_range), top='confirmed', bottom=0,
           width=0.2, color=colors[1], source = source, name='confirmed', legend_label="Confirmed")
    
    p.vbar(x=dodge('x', 0.25, range=p.x_range), top='death', bottom=0,
           width=0.2, color=colors[2], source = source, name='death', legend_label="Death")
    
    #p.x_range.range_padding = 0.1
    p.xaxis.major_label_orientation = -1.3
    p_g.xaxis.major_label_orientation = -1.3
    
    p.legend.location = "top_left"
    p.legend.orientation = "horizontal"
    p_g.legend.location = "top_left"
    p_g.legend.orientation = "horizontal"
    
    #Update Data
    def update():
        print('Update')
        new_data = select_region()
        new_data_g = global_virus()
        source.data = dict( x = new_data['Month'], recovered = new_data['Recovered'], 
                           confirmed =new_data['Confirm'], death = new_data['Death'],
                           status = s[:len(new_data['Month'])])
        
        source_g.data = dict( x = new_data_g['Month'], recovered = new_data_g['Recovered'], 
                           confirmed =new_data_g['Confirm'], death = new_data_g['Death'],
                           status = s[:len(new_data_g['Month'])])
        
        p.title.text ='Region %s Covid-19' %region.value
    
    #Set Control
    controls = [region, slider_month]
    for control in controls :
        control.on_change('value', lambda attr, old, new: update())
        
    RB.on_change('active', lambda attr, old, new: update())
    l = layout(children=[region,slider_month, RB, p, p_g] , sizing_mode='scale_width')
    
    #inputs = column(*controls, width=200, height=500)
    #inputs.sizing_mode='fixed'
    #l = layout([inputs, p], sizing_mode="scale_both")
    
    doc.add_root(l)
show(modify_doc)

Loading BokehJS ...

Afghanistan
{'Confirm': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 5, 7, 7, 7, 11, 16, 21, 22, 22, 22, 24, 24, 40, 40, 74, 84, 94, 110, 110, 120, 170, 174, 237, 273, 281, 299, 349, 367, 423, 444, 484, 521, 555, 607, 665, 714, 784, 840], 'Recovered': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 5, 5, 10, 10, 10, 15, 18, 18, 29, 32, 32, 32, 32, 32, 40, 43, 54], 'Death': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 4, 4, 4, 4, 4, 4, 4, 6, 6, 7, 7, 11, 14, 14, 15, 15, 18, 18, 21, 23, 25, 30], 'Month': ['1/22/20', '1/23/20', '1/24/20', '1/25/20', '1/26/20', '1/27/20', '1/28/20', '1/29/